In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My\ Drive/Colab/IDL_Final_Project/new

/content/gdrive/.shortcut-targets-by-id/1Sr9hCVVWERdsdT-sePtsjuVIxzvSrDI1/Colab/IDL_Final_Project/new


In [ ]:
# %cp -av train train_copy

## Installing Sox

In [ ]:
# audio converters
# !apt update && apt-get install ffmpeg mpg123

# sox package for adjusting sample rate.
!apt-get install libsox-fmt-all libsox-dev sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-ao
  libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3
Suggested packages:
  libaudio2 file
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-dev libsox-fmt-all
  libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss
  libsox-fmt-pulse libsox3 sox
0 upgraded, 18 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,267 kB of archives.
After this operation, 9,153 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/u

## Install VAD

In [ ]:
!pip install webrtcvad

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71362 sha256=29dde35e2a868592608f97aea2cafc79300005db81eac0cf835335ae72d1b2c4
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
Successfully built webrtcvad


In [ ]:
!pip install Resemblyzer

     |████████████████████████████████| 15.7MB 1.2MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=207623d4c57c69e0dc8d042ebb80187af939fdc2cb34dbbb198040a884994ce7
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built typing


## Imports

In [ ]:
# from scipy.ndimage.morphology import binary_dilation
# from resemblyzer.hparams import *
from pathlib import Path
from typing import Optional, Union
import numpy as np
# import webrtcvad
import librosa
import struct
import IPython.display as ipd

#https://colab.research.google.com/drive/1fXJ_YUWACs0w8ohORFijuWo4_pCLMIEK#scrollTo=CrH9g0n1hzpK
import wave
import os
import shlex
import subprocess
import sys

In [ ]:
ipd.Audio('new/test_audio_new/20100310_124750_4615.wav')

In [ ]:
ipd.Audio('new/test_audio_new/20100310_115237_4614.wav')

## Using Sox

In [ ]:
ipd.Audio('noise-audio.wav')

Get noise part from audio

In [21]:
# !sox mixer/1-channel-wav/20090713_122937_5.wav noise-audio.wav trim 0 0.900

sox_cmd = 'sox mixer/1-channel-wav/20090713_122937_5.wav noise-audio.wav trim 0 0.900'
try:
  # output = 
  subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
  raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
except OSError as e:
  raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

Generate a noise profile in sox:

In [22]:
# Generate a noise profile in sox:
# sox noise-audio.wav -n noiseprof noise.prof

sox_cmd = 'sox noise-audio.wav -n noiseprof noise.prof'
try:
  subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
  raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
except OSError as e:
  raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

Remove noise / Clean the noise from the audio

In [25]:
sox_cmd = 'sox mixer/1-channel-wav/20090713_122937_5.wav audio-clean.wav noisered noise.prof 0.21'
try:
  subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
  raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
except OSError as e:
  raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

In [ ]:
ipd.Audio('mixer/1-channel-wav/20090713_122937_5.wav')

Output hidden; open in https://colab.research.google.com to view.

In [26]:
ipd.Audio('audio-clean.wav')

Output hidden; open in https://colab.research.google.com to view.

In [33]:
sox_cmd = 'sox test_audio_new/20100310_115237_4614.wav no_silence.wav silence 2 0.1 1% -1 0.1 1%'
try:
  subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
  raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
except OSError as e:
  raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

In [35]:
ipd.Audio('test_audio_new/20100310_115237_4614.wav')

In [34]:
ipd.Audio('no_silence.wav')

In [ ]:
def convert_samplerate(audio_path):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate 16000 --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(audio_path)
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

    return 16000, np.frombuffer(output, np.int16)

## PySox

In [ ]:
!pip install pysox

     |████████████████████████████████| 174kB 12.2MB/s 
  ERROR: Failed building wheel for pysox
  Running setup.py clean for pysox
Failed to build pysox
    Running setup.py install for pysox ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-yk7x0zj3/pysox/setup.py'"'"'; __file__='"'"'/tmp/pip-install-yk7x0zj3/pysox/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-nfbio9xy/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


## Code

### trim_long_silences & normalize_volume

In [ ]:
def trim_long_silences(wav):
    """
    Ensures that segments without voice in the waveform remain no longer than a 
    threshold determined by the VAD parameters in params.py.

    :param wav: the raw waveform as a numpy array of floats 
    :return: the same waveform with silences trimmed away (length <= original wav length)
    """
    # Compute the voice detection window size
    samples_per_window = (vad_window_length * sampling_rate) // 1000
    
    # Trim the end of the audio to have a multiple of the window size
    wav = wav[:len(wav) - (len(wav) % samples_per_window)]
    
    # Convert the float waveform to 16-bit mono PCM
    pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))
    
    # Perform voice activation detection
    voice_flags = []
    vad = webrtcvad.Vad(mode=3)
    for window_start in range(0, len(wav), samples_per_window):
        window_end = window_start + samples_per_window
        voice_flags.append(vad.is_speech(pcm_wave[window_start * 2:window_end * 2],
                                         sample_rate=sampling_rate))
    voice_flags = np.array(voice_flags)
    
    # Smooth the voice detection with a moving average
    def moving_average(array, width):
        array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
        ret = np.cumsum(array_padded, dtype=float)
        ret[width:] = ret[width:] - ret[:-width]
        return ret[width - 1:] / width
    
    audio_mask = moving_average(voice_flags, vad_moving_average_width)
    audio_mask = np.round(audio_mask).astype(np.bool)
    
    # Dilate the voiced regions
    audio_mask = binary_dilation(audio_mask, np.ones(vad_max_silence_length + 1))
    audio_mask = np.repeat(audio_mask, samples_per_window)
    
    return wav[audio_mask == True]


def normalize_volume(wav, target_dBFS, increase_only=False, decrease_only=False):
    if increase_only and decrease_only:
        raise ValueError("Both increase only and decrease only are set")
    rms = np.sqrt(np.mean((wav * int16_max) ** 2))
    wave_dBFS = 20 * np.log10(rms / int16_max)
    dBFS_change = target_dBFS - wave_dBFS
    if dBFS_change < 0 and increase_only or dBFS_change > 0 and decrease_only:
        return wav
    return wav * (10 ** (dBFS_change / 20))

### Preprocess

In [ ]:
int16_max = (2 ** 15) - 1


def preprocess_wav(fpath_or_wav: Union[str, Path, np.ndarray], source_sr: Optional[int]=None):
    """
    Applies preprocessing operations to a waveform either on disk or in memory such that  
    The waveform will be resampled to match the data hyperparameters.

    :param fpath_or_wav: either a filepath to an audio file (many extensions are supported, not 
    just .wav), either the waveform as a numpy array of floats.
    :param source_sr: if passing an audio waveform, the sampling rate of the waveform before 
    preprocessing. After preprocessing, the waveform'speaker sampling rate will match the data 
    hyperparameters. If passing a filepath, the sampling rate will be automatically detected and 
    this argument will be ignored.
    """
    # Load the wav from disk if needed
    if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
        wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
    else:
        wav = fpath_or_wav
    
    # Resample the wav
    if source_sr is not None:
        wav = librosa.resample(wav, source_sr, sampling_rate)
        
    # Apply the preprocessing: normalize volume and shorten long silences 
    wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)
    wav = trim_long_silences(wav)
    
    return wav

In [ ]:
FILENAME = 'mixer/1-channel-wav/20090715_161726_18.wav'
# sample_rate = 8000
wav = preprocess_wav(FILENAME)

In [ ]:
type(wav).__name__

'ndarray'

In [ ]:
# import wave
# import numpy as np

# Save channel to a separate file
# outwav = wave.open(fn, 'w')
# outwav.setparams(wav.getparams())
# outwav.setnchannels(1)
# outwav.writeframes(ch_data.tostring())
# outwav.close()

librosa.output.write_wav('20090715_161726_18.wav', wav, sample_rate)

In [ ]:
ipd.Audio('20090715_161726_18.wav')